ใน ep นี้เราจะเรียนรู้

# 0. Install

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Install Library ที่จำเป็น

In [2]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


เช็ค GPU

In [3]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai และ fastai.text 

In [0]:
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.callbacks.mem import *

In [0]:
# import fastai.utils.collect_env
# fastai.utils.collect_env.show_install()

สำหรับ [Google Colab](https://www.bualabs.com/archives/1687/what-is-colab-open-jupyter-notebook-in-github-on-google-colab-create-open-in-colab-button-colab-ep-1/) เราจะกำหนด path ที่เก็บ DataSet และ Mount Google Drive 

In [17]:
dataset = '26i-giga-fren'

## Google Colab
config_path = Path('/content/drive')
data_path_base = Path('/content/drive/My Drive/datasets/')

# ## VM
# config_path = Path(os.getenv("HOME"))
# data_path_base = Path(os.getenv("HOME")'/datasets/')

data_path = data_path_base/dataset

from google.colab import drive
drive.mount(str(config_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Dataset

ในเคสนี้เราจะใช้ giga-fren คือ Giga-word corpus สำหรับแปลภาษาระหว่างภาษาฝรั่งเศส-ภาษาอังกฤษ (French-English) จากงาน WMT2010 รวบรวมโดย Chris Callison-Burch

In [7]:
data_path

PosixPath('/content/drive/My Drive/datasets/26i-giga-fren')

ดูว่ามีไฟล์อะไรบ้าง

In [8]:
path = data_path/'giga-fren'
path.ls()

[PosixPath('/content/drive/My Drive/datasets/26i-giga-fren/giga-fren/giga-fren.release2.fixed.en'),
 PosixPath('/content/drive/My Drive/datasets/26i-giga-fren/giga-fren/questions_easy.csv'),
 PosixPath('/content/drive/My Drive/datasets/26i-giga-fren/giga-fren/.ipynb_checkpoints'),
 PosixPath('/content/drive/My Drive/datasets/26i-giga-fren/giga-fren/databunch-small-questions.pkl')]

# 3. Preprocess

In [0]:
def seq2seq_collate(samples, pad_idx=1, pad_first=True, backwards=False):
    samples = to_data(samples)
    max_len_x, max_len_y = max([len(s[0]) for s in samples]), max([len(s[1]) for s in samples])
    res_x = torch.zeros(len(samples), max_len_x).long() + pad_idx
    res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx

    if backwards: pad_first = not pad_first
    for i, s in enumerate(samples): 
        if pad_first:
            res_x[i, -len(s[0]):], res_y[i, -len(s[1]):] = LongTensor(s[0]), LongTensor(s[1])
        else:
            res_x[i, :len(s[0]):], res_y[i, :len(s[1]):] = LongTensor(s[0]), LongTensor(s[1])
    if backwards: res_x, res_y = res_x.flip(1), res_y.flip(1)
    return res_x, res_y

In [0]:
class Seq2SeqDataBunch(TextDataBunch):
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', 
               bs:int=32, val_bs:int=None, pad_idx=1, dl_tfms=None, 
               pad_first=False, device:torch.device=None, no_check:bool=False, 
               backwards:bool=False, **dl_kwargs) -> DataBunch:

        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(seq2seq_collate, pad_idx=pad_idx, 
                             pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, 
                                       key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, collate_fn=collate_fn, no_check=no_check)


In [0]:
class Seq2SeqTextList(TextList):
    _bunch = Seq2SeqDataBunch
    _label_cls = TextList

โหลด DataBunch ที่สร้างไว้ใน [ep ที่แล้ว Sequence to Sequence RNN Model](https://www.bualabs.com/archives/3134/what-is-sequence-to-sequence-neural-translate-machine-translation-french-to-english-sequence-to-sequence-model-rnn-gru-teacher-forcing-nlp-ep-10/) ขึ้นมา

In [0]:
data = load_data(path, 'databunch-small-questions.pkl')

# 4. Model

## 4.1 Embedding

โหลด Embedding ที่สร้างไว้ใน [ep ที่แล้ว Sequence to Sequence RNN Model](https://www.bualabs.com/archives/3134/what-is-sequence-to-sequence-neural-translate-machine-translation-french-to-english-sequence-to-sequence-model-rnn-gru-teacher-forcing-nlp-ep-10/) ขึ้นมา

In [0]:
model_path = config_path/'My Drive/models'

In [0]:
emb_enc = torch.load(model_path/'fr_emb.pth')
emb_dec = torch.load(model_path/'en_emb.pth')

## 4.2 Loss, Metrics and Callback

Loss Function, Metrics และ Teacher Forcing Callback จาก ep ที่แล้ว

In [0]:
def seq2seq_loss(out, targ, pad_idx=1):
    bs, targ_len = targ.size()
    _, out_len, vs = out.size()
    if targ_len > out_len: out = F.pad(out, (0, 0, 0, targ_len-out_len, 0, 0), value=pad_idx)
    if out_len > targ_len: targ = F.pad(targ, (0, out_len-targ_len, 0, 0), value=pad_idx)
    return CrossEntropyFlat()(out, targ)

In [0]:
def seq2seq_acc(out, targ, pad_idx=1):
    bs, targ_len = targ.size()
    _, out_len, vs = out.size()
    if targ_len > out_len: out = F.pad(out, (0, 0, 0, targ_len-out_len, 0, 0), value=pad_idx)
    if out_len > targ_len: targ = F.pad(targ, (0, out_len-targ_len, 0, 0), value=pad_idx)
    out = out.argmax(2)
    return (out == targ).float().mean()

In [0]:
class NGram():
    def __init__(self, ngram, max_n=50000): self.ngram, self.max_n = ngram, max_n
    def __eq__(self, other): 
        if len(self.ngram) != len(other.ngram): return False
        return np.all(np.array(self.ngram) == np.array(other.ngram))
    def __hash__(self): return int(sum([o * self.max_n**i for i, o in enumerate(self.ngram)]))

In [0]:
def get_grams(x, n, max_n=5000):
    return x if n==1 else [NGram(x[i:i+n], max_n=max_n) for i in range(len(x)-n+1)]

In [0]:
def get_correct_ngrams(pred, targ, n, max_n=5000): 
    pred_grams, targ_grams = get_grams(pred, n, max_n=max_n), get_grams(targ, n, max_n=max_n)
    pred_cnt, targ_cnt = Counter(pred_grams), Counter(targ_grams)
    return sum([min(c, targ_cnt[g]) for g, c in pred_cnt.items()]), len(pred_grams)

In [0]:
class CorpusBLEU(Callback):
    def __init__(self, vocab_sz):
        self.vocab_sz = vocab_sz
        self.name = 'bleu'

    def on_epoch_begin(self, **kwargs):
        self.pred_len, self.targ_len, self.corrects, self.counts = 0, 0, [0]*4, [0]*4
    
    def on_batch_end(self, last_output, last_target, **kwargs):
        last_output = last_output.argmax(dim=-1)
        for pred, targ in zip(last_output.cpu().numpy(), last_target.cpu().numpy()):
            self.pred_len += len(pred)
            self.targ_len += len(targ)
            for i in range(4):
                c, t = get_correct_ngrams(pred, targ, i+1, max_n=self.vocab_sz)
                self.corrects[i] += c
                self.counts[i] += t
    
    def on_epoch_end(self, last_metrics, **kwargs):
        precs = [c/t for c, t in zip(self.corrects, self.counts)]
        len_penalty = exp(1 - self.targ_len / self.pred_len) if self.pred_len < self.targ_len else 1
        bleu = len_penalty * ((precs[0] * precs[1] * precs[2] * precs[3]) ** 0.25)
        return add_metrics(last_metrics, bleu)

In [0]:
def get_predictions(learn, ds_type=DatasetType.Valid):
    learn.model.eval()
    inputs, targets, outputs = [], [], []
    with torch.no_grad():
        for xb, yb in progress_bar(learn.dl(ds_type)):
            out = learn.model(xb)
            for x, y, z in zip(xb, yb, out):
                inputs.append(learn.data.train_ds.x.reconstruct(x))
                targets.append(learn.data.train_ds.y.reconstruct(y))
                outputs.append(learn.data.train_ds.y.reconstruct(z.argmax(1)))
    return inputs, targets, outputs

In [0]:
class TeacherForcing(LearnerCallback):
    def __init__(self, learn, end_epoch):
        super().__init__(learn)
        self.end_epoch = end_epoch
    
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        if train: return {'last_input': [last_input, last_target]}

    def on_epoch_begin(self, epoch, **kwargs):
        self.learn.model.pr_force = 1 - epoch/self.end_epoch

## 4.3 New Model with Attention

# 7. สรุป

* เราได้เรียนรู้


ตารางสรุป ผลลัพธ์การเทรนโมเดลแบบต่าง ๆ

model            | train_loss | valid_loss | seq2seq_acc | bleu
-------------------|----------|----------|----------|----------
seq2seq            | 2.823382 | 3.826643 | 0.427117 | 0.322879
\+ teacher forcing | 2.678760 |	3.732396 | 0.439311 | 0.332833
\+ attention       | 1.452292 | 3.420485 | 0.498205 | 0.413232
transformer        | 1.913152 | 2.349686 | 0.781749 | 0.612880

# Credit

* https://www.youtube.com/watch?v=IfsjMg4fLWQ&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=12
* https://www.bualabs.com/archives/3134/what-is-sequence-to-sequence-neural-translate-machine-translation-french-to-english-sequence-to-sequence-model-rnn-gru-teacher-forcing-nlp-ep-10/
* https://www.bualabs.com/archives/3103/what-is-recurrent-neural-network-rnn-pytorch-gru-nlp-ep-9/
* https://www.bualabs.com/archives/3087/sentiment-classification-deep-learning-imdb-movie-reviews-positive-negative-deep-neural-network-awd-lstm-ulmfit-nlp-ep-8/
* https://www.bualabs.com/archives/3060/what-is-n-gram-sentiment-classification-imdb-movie-review-naive-bayes-logistic-regression-nlp-ep-6/
* http://opus.nlpl.eu/giga-fren.php
* https://arxiv.org/abs/1409.3215
* https://ieeexplore.ieee.org/document/6795228/
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
* http://www.statmt.org/wmt15/translation-task.html
* https://fasttext.cc/docs/en/crawl-vectors.html
* https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213
* https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21
* https://github.com/fastai/fastai_docs/blob/master/dev_course/dl2/translation.ipynb
* 